# Google Cloud Dataflow

Table of contents:

- [Beam Programming](#beam-programming)
- [Building a Simple Data Transformation Pipeline](#building-a-simple-data-transformation-pipeline)


Google Cloud Dataflow provides a serverless, parallel and distributed infrastructure for running jobs for batch and stream data processing.
One of the core strengths of Dataflow is its ability to almost seamlessly handle the switch from processing of batch historical data to streaming datasets while elegantly taking into consideration the perks of streaming processing such as windowing.
Dataflow is a major component for building an end-to-end ML production pipeline on GCP.

<a id="beam-programming"></a>

## Beam Programming
Apache Beam provides a set of broad concepts to simplify the process of building a transformation pipeline for distributed batch and stream jobs.

- **A Pipeline:** A Pipeline object wraps the entire operation and prescribes the transformation process by defining the input data source to the pipeline, how that data will be transformed and where the data will be written.
- **A PCollection:** A PCollection is used to define a data source. The data source can either be bounded or unbounded. A bounded data source referes to batch or historical data, whereas an unbounded data source refers to streaming data.
- **A PTransform:** PTransforms refers to a particular transformation task carried out on one or more PCollections in the pipeline. A number of core Beam transforms include:
  - ParDo: for parallel processing.
  - GroupByKey: for processing collections of key/value pairs.
  - CoGroupByKey: for a relational join of two or more key/value PCollections with the same key type.
  - Combine: for combining collections of elements or values in your data.
  - Flatten: for merging multiple PCollection objects.
  - Partition: splits a single PCollection into smaller collections. 
- **I/O Transforms:** These are PTransforms that read or write data to different external storage systems.

<div style="display: inline-block;width: 100%;">
<img src="ieee-ompi/dataflow-sequential-transform.png" style="float:left;" alt="A Simple Linear Pipeline with Sequential Transforms." height=90% width=90% />
</div>

<a id="building-a-simple-data-transformation-pipeline"></a>

## Building a Simple Data Transformation Pipeline

In [1]:
# install the apache beam library and other important setup packages.
# restart the session after installing apache beam.

In [10]:
%%bash
source activate py2env
pip uninstall -y google-cloud-dataflow
conda install -y pytz==2018.4
pip install apache-beam[gcp]

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/py2env

  added / updated specs: 
    - pytz==2018.4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.12.5  |                0         123 KB  defaults

The following packages will be UPDATED:

    ca-certificates: 2018.03.07-0 defaults --> 2018.12.5-0 defaults


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
  Using cached https://files.pythonhosted.org/packages/d4/3d/90aa15779e884feebae4b0c26cad6f52cd4040397a94deb58dad9c8b7300/apache_beam-2.9.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c3/f1/e61d6dfe6c1768ed2529761a68f70939e2569da043e9f15a8d84bf56cadf/pydot-1.2.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/fd/ce/aa4a385e3e9

Skipping google-cloud-dataflow as it is not installed.


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.2

Please update conda by running

    $ conda update -n base -c defaults conda


ca-certificates-2018 | 123 KB    | ########## | 100% 
google-cloud-bigquery 1.6.1 has requirement google-api-core<2.0.0dev,>=1.0.0, but you'll have google-api-core 0.1.4 which is incompatible.
googledatastore 7.0.1 has requirement httplib2<0.10,>=0.9.1, but you'll have httplib2 0.11.3 which is incompatible.


In [2]:
# import relevant libraries
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

In [ ]:
def run(project, source_bucket, target_bucket):
    import csv

    options = {
        'staging_location': 'gs://df-bucket-sb/staging',
        'temp_location': 'gs://df-bucket-sb/temp',
        'job_name': 'dataflow-crypto',
        'project': project,
        'max_num_workers': 24,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True,
        'runner': 'DataflowRunner'
      }
    options = beam.pipeline.PipelineOptions(flags=[], **options)
    
    crypto_dataset = 'gs://{}/crypto-markets.csv'.format(source_bucket)
    processed_ds = 'gs://{}/transformed-crypto-bitcoin'.format(target_bucket)

    pipeline = beam.Pipeline(options=options)

    # 0:slug, 3:date, 5:open, 6:high, 7:low, 8:close
    rows = (
        pipeline |
            'Read from bucket' >> ReadFromText(crypto_dataset) |
            'Tokenize as csv columns' >> beam.Map(lambda line: next(csv.reader([line]))) |
            'Select columns' >> beam.Map(lambda fields: (fields[0], fields[3], fields[5], fields[6], fields[7], fields[8])) |
            'Filter bitcoin rows' >> beam.Filter(lambda row: row[0] == 'bitcoin')
        )
        
    combined = (
        rows |
            'Write to bucket' >> beam.Map(lambda (slug, date, open, high, low, close): '{},{},{},{},{},{}'.format(
                slug, date, open, high, low, close)) |
            WriteToText(
                file_path_prefix=processed_ds,
                file_name_suffix=".csv", num_shards=2,
                shard_name_template="-SS-of-NN",
                header='slug, date, open, high, low, close')
        )

    pipeline.run()